In [1]:
import tensorflow as tf
import os
import zipfile
from os import path, getcwd, chdir

path = f"{getcwd()}/../tmp2/happy-or-sad.zip"

zip_ref = zipfile.ZipFile(path, 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

In [6]:
def train_happy_sad_model():

    DESIRED_ACCURACY = 0.999

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('acc')>0.999):
                print("\n Reached 0.999 accuracy, so cancelling training!")
                self.model.stop_training=True
         
    callbacks = myCallback()
    
    # This Code Block should Define and Compile the Model. Please assume the images are 150 X 150 in your implementation.
    model = tf.keras.models.Sequential([
       
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
    
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
    
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['accuracy'])
        

    # This code block should create an instance of an ImageDataGenerator called train_datagen 
    # And a train_generator by calling train_datagen.flow_from_directory

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    train_datagen = ImageDataGenerator(rescale=1/255)

    # target_size of 150 X 150.
    train_generator = train_datagen.flow_from_directory(
        
        "/tmp/h-or-s",
        target_size=(150,150),
        batch_size=128,
        class_mode='binary')
    # Expected output: 'Found 80 images belonging to 2 classes'

    # This code block should call model.fit_generator and train for
    # a number of epochs.
    # model fitting
    history = model.fit_generator(
          
          train_generator,
          steps_per_epoch=8,  
          epochs=15,
          verbose=1, callbacks=[callbacks])
    # model fitting
    return history.history['acc'][-1]

In [7]:
train_happy_sad_model()

Found 80 images belonging to 2 classes.
Epoch 1/15
8/8 [==============================] - 4s 547ms/step - loss: 1.8682 - acc: 0.5531
Epoch 2/15
8/8 [==============================] - 2s 302ms/step - loss: 0.3522 - acc: 0.8438
Epoch 3/15
8/8 [==============================] - 2s 312ms/step - loss: 0.0983 - acc: 0.9719
Epoch 4/15
8/8 [==============================] - 3s 314ms/step - loss: 0.0357 - acc: 0.9906
Epoch 5/15
7/8 [=========================>....] - ETA: 0s - loss: 0.0093 - acc: 1.0000
 Reached 0.999 accuracy, so cancelling training!
8/8 [==============================] - 3s 313ms/step - loss: 0.0088 - acc: 1.0000


1.0